In [1]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB
from ipynb.fs.defs.MMRWPC import MMRWPC
import time
from joblib import Parallel, delayed
import os
from ipynb.fs.defs.GraphReader import GraphReader

In [5]:
class MMPC_Testing():
    
    def __init__(self, mmpc_model, model_name, testing_dataset_route, log_route, selected_nodes_route,
                 graph_info_route, n_forks):
        
        self._mmpc_model = mmpc_model
        self._model_name = model_name
        self._testing_dataset_route = testing_dataset_route
        self._log_route = log_route
        self._n_forks = n_forks
        self._graph_reader = GraphReader(graph_info_route)
        self._files_dict = None
        self._results = None
        preselected_nodes = []
        try:
            ff = open(log_route+selected_nodes_route,'r')
            for e in ff.read().split('\n'):
                spl_line = e.split(';')
                if (len(spl_line[0])==0):
                    continue
                preselected_nodes.append(spl_line[0])
            ff.close()
        except:
            print("Error opening preselected nodes file")
            return
        
        self._selected_nodes = preselected_nodes
        self.__init_files()
        print("MMPC_Testing object succesfully created.")
        
        return
    
    def __init_files(self):
        
        files_dict = {}
        for filename in os.listdir(self._testing_dataset_route):
            temp_df = pd.read_csv(
                self._testing_dataset_route+
                filename,delimiter='  ',header=None, engine='python')
            graph_heads = []
            for k in range(0,len(temp_df.columns)):
                graph_heads.append('Node'+str(k+1))
            temp_df.columns = graph_heads
            files_dict[filename] = temp_df
            print(filename)
        
        self._files_dict = files_dict
        return
    
    def getResults(self):
        if self._results is None:
            print("No calculated results yet.")
            
            return []
        
        return list(self._results)
    
    
    def __log_time_results(self, pc_list, file_dict):
        ff = open(self._log_route+'time_results_'+self._model_name+'_'+file_dict, 'w')
        for e in pc_list:
            ff.write(e['dataset']+';')
            ff.write(e['TargetNode']+';')
            ff.write(str(e['time'])+';')
            ff.write(str(e['pc-distance'])+';')
            for node in e['PC']:
                ff.write(node['name']+',')
            ff.write('\n')
        ff.close()
    
    def __evaluate(self, PCs):
        
        processed_pc_list = []
        for file_dict in self._files_dict:
            nodes_to_process = [x for x in PCs if x['dataset'] == file_dict]
            self._mmpc_model.fit(self._files_dict[file_dict])
            pcs_index = [int(x['TargetNode'].split('@')[0].split('Node')[1])-1 for x in nodes_to_process]
            pc_list = Parallel(n_jobs=len(nodes_to_process))(map(delayed(self._mmpc_model.evaluate),pcs_index))
            
            s_processed_pc_list = []
            for k in range(0,len(nodes_to_process)):
                pob_dict = {}
                pob_dict['dataset'] = file_dict
                pob_dict['TargetNode'] = nodes_to_process[k]['TargetNode']
                print(pob_dict['TargetNode'])
                pob_dict['PC'] = pc_list[k]['CPC']
                pob_dict['time'] = pc_list[k]['time']
                pob_dict['pc-distance'] = self._graph_reader.getDistance(pcs_index[k],[x['name'].split('@')[0] for x in pob_dict['PC']])
                s_processed_pc_list.append(pob_dict)
            
            self.__log_time_results(s_processed_pc_list,file_dict)
            
            for e in s_processed_pc_list:
                processed_pc_list.append(e)
                
        return processed_pc_list
    
    def test_model(self):
        PCs = self.__getNodeDicts()
        model_pcs = self.__evaluate(PCs)
#         scored_pcs = Parallel(n_jobs=10)(map(delayed(self.__scorePC),PCs))
#         self._results = scored_pcs
        self._results = model_pcs
        
        return
    
    def __getNodeDicts(self):
        """
        Create list of parameters to pass
        for a parallel execution including
        the file to examine and the node to examine
        """
        pob_dicts = []
        for node in self._selected_nodes:
            for filename in os.listdir(self._testing_dataset_route):
                pob_dict = {}
                pob_dict['dataset'] = filename
                pob_dict['TargetNode'] = node
                pob_dicts.append(pob_dict)
        
        return pob_dicts
    
    def __scorePC(self, PobDict):
        X_train_df = self._files_dict[PobDict['dataset']]

        #X_train, X_test, Y_train, Y_test = train_test_split(X_train_df, X_train_df[TargetEvalNode], test_size=0.1, random_state=0)

        X_train = X_train_df
        Y_train = X_train_df[PobDict['TargetNode']]

        clf = GaussianNB()
        clf.fit(X_train[PobDict['PC']],Y_train)
        
        

        PobDict['class_precisions']=[]
        for filename in [x for x in os.listdir(self._testing_dataset_route) if x.split('_')[1]==PobDict['dataset'].split('_')[1]]:
            if (filename == PobDict['dataset']):
                continue
            #print("*****")
            #print()
            #print("Testing on dataset: "+filename)
            X_test = files_dict[filename]
            X_test.columns = graph_heads


            # get score values
            precision = []
            values_counter=set(X_train_df[PobDict['TargetNode']])
            for val in values_counter:
                #Y_pred_c = clf.predict(X_train_df.query( TargetEvalNode+'== '+str(i))[PobDict['supercpc']][-100:])
                if (len(X_test.query( PobDict['TargetNode']+'== '+str(val)))==0):
                    precision.append([val,0,0])
                    continue
                Y_pred_c = clf.predict(X_test.query( PobDict['TargetNode']+'== '+str(val))[PobDict['PC']])
                precision.append([val,len(Y_pred_c),Y_pred_c.tolist().count(val)/len(Y_pred_c)])

            #for e in range(0,len(precision)):
                #print('precision for class '+str(precision[e][0])+' with '+str(precision[e][1])+' samples in dataset : '+str(precision[e][2]))

            #print("G-mean score: "+str(g_mean(precision)))
            #print("*****")
            #print()
            #print(clf.score(X_test[PobDict['supercpc'][:10]],Y_test))
            PobDict['class_precisions'].append([precision,filename])
        print("Exiting Candidate NB Scoring function")
        return PobDict
    

In [ ]:
testing_obj  = MMPC_Testing(MMRWPC(0.05,0.0,1),
                            'MMRWPC',
                            '/home/sdelrio/alarm_datasets/Alarm10/', 
                            '/home/sdelrio/alarm_datasets/log_results/',
                            'mmpc_partitioned/alarm10_binary_nodes_selected.txt',
                            '/home/sdelrio/Alarm10_graph.txt',
                            10)

Random Walk successful instantiation with:
-Alpha: 0.05
-Delta: 0.0
-Max Neighbours per run: 1
Alarm10_s1000_v4.txt
Alarm10_s500_v7.txt
Alarm10_s5000_v1.txt
Alarm10_s5000_v9.txt
Alarm10_s500_v6.txt
Alarm10_s1000_v8.txt
Alarm10_s5000_v8.txt
Alarm10_s1000_v1.txt
Alarm10_s1000_v5.txt
Alarm10_s500_v9.txt
Alarm10_s1000_v6.txt
Alarm10_s500_v2.txt
Alarm10_s5000_v5.txt
Alarm10_s500_v1.txt
Alarm10_s5000_v4.txt
Alarm10_s1000_v7.txt
Alarm10_s500_v4.txt
Alarm10_s5000_v2.txt
Alarm10_s1000_v3.txt
Alarm10_s1000_v2.txt
Alarm10_s5000_v6.txt
Alarm10_s500_v3.txt
Alarm10_s1000_v10.txt
Alarm10_s5000_v7.txt
Alarm10_s500_v5.txt
Alarm10_s500_v10.txt
Alarm10_s5000_v3.txt
Alarm10_s500_v8.txt
Alarm10_s5000_v10.txt
Alarm10_s1000_v9.txt
MMPC_Testing object succesfully created.


In [ ]:
testing_obj.test_model()

Succesful instantiation of Dep object
Succesfully fitted array of 370 columns
and 1000 rows per column.
Entering Phase I
MMPC_beggining: 
369

Universe actual size:

5
CPC size before filtering: 1
CPC size after filtering: 1
['Node76@4']

Universe actual size:
4
CPC size before filtering: 2
Size set for community selection phase will be of maximum: 2
['Node76@4', 'Node96@4']
n_set of length: 2
CPC size after filtering: 2
['Node96@4', 'Node76@4']

Universe actual size:
3
CPC size before filtering: 3
Size set for community selection phase will be of maximum: 3
['Node76@4', 'Node96@4', 'Node39@4']
n_set of length: 3
CPC size after filtering: 3
['Node39@4', 'Node96@4', 'Node76@4']
Entering Phase I
MMPC_beggining: 
369
Universe actual size:['Node89@3', 'Node96@4', 'Node103@3', 'Node92@3']

1
CPC size before filtering: 4
Size set for community selection phase will be of maximum: 3
['Node76@4', 'Node96@4', 'Node39@4', 'Node78@4']
n_set of length: 4
CPC size after filtering: 4
['Node78@4', 'No

['Node127@3', 'Node304@3']
n_set of length: 2
CPC size after filtering: 2
['Node304@3', 'Node127@3']

Universe actual size:
2
CPC size before filtering: 3
Size set for community selection phase will be of maximum: 3
['Node127@3', 'Node76@4', 'Node304@3']
n_set of length: 3
CPC size after filtering: 3
['Node304@3', 'Node76@4', 'Node127@3']

Universe actual size:
1
CPC size before filtering: 4
Size set for community selection phase will be of maximum: 3
['Node127@3', 'Node76@4', 'Node304@3', 'Node229@3']
n_set of length: 4
CPC size after filtering: 4
Entering Phase I
MMPC_beggining: 
369

Universe actual size:

3
CPC size before filtering: 1
CPC size after filtering: 1
['Node35@3']

Universe actual size:
1
CPC size before filtering: 2
Size set for community selection phase will be of maximum: 2
['Node35@3', 'Node130@3']
n_set of length: 2
CPC size after filtering: 2
['Node130@3', 'Node35@3']

Universe actual size:
0
CPC size before filtering: 3
Size set for community selection phase will

['Node184@2', 'Node183@3', 'Node181@3', 'Node180@3']
n_set of length: 4
CPC size after filtering: 3
['Node180@3', 'Node183@3', 'Node184@2']
Entering Phase I
MMPC_beggining: 
369
Universe actual size:
1
CPC size before filtering: 4
Size set for community selection phase will be of maximum: 3
['Node180@3', 'Node183@3', 'Node184@2', 'Node160@3']
n_set of length: 4
CPC size after filtering: 4
['Node160@3', 'Node184@2', 'Node183@3', 'Node180@3']

Universe actual size:

13
CPC size before filtering: 6
Size set for community selection phase will be of maximum: 4

Universe actual size:
8
CPC size before filtering: 6
Size set for community selection phase will be of maximum: 4

Universe actual size:
6
CPC size before filtering: 5
Size set for community selection phase will be of maximum: 4
['Node155@3', 'Node156@3', 'Node215@2', 'Node5@3', 'Node113@4']
n_set of length: 5
Entering Phase I
MMPC_beggining: 
369CPC size after filtering: 5

['Node113@4', 'Node5@3', 'Node215@2', 'Node156@3', 'Node155

Entering Phase I
['Node16@3', 'Node355@4', 'Node168@2', 'Node266@3']
CPC size after filtering: 4
['Node266@3', 'Node168@2', 'Node355@4', 'Node16@3']
MMPC_beggining: 
369
Entering Phase I

MMPC_beggining: 
369
Universe actual size:
6
CPC size before filtering: 5
Size set for community selection phase will be of maximum: 4
['Node183@3', 'Node182@3', 'Node181@3', 'Node180@3', 'Node159@2']
n_set of length: 5
['Node12@3', 'Node5@3', 'Node7@3', 'Node8@3', 'Node13@3', 'Node15@3', 'Node21@4', 'Node74@2']

Universe actual size:n_set of length: 8
CPC size after filtering: 3
['Node159@2', 'Node182@3', 'Node183@3']

Universe actual size:CPC size before filtering: 4

0
Size set for community selection phase will be of maximum: 3
CPC size before filtering: 3

['Node159@2', 'Node182@3', 'Node183@3', 'Node158@3']
5
n_set of length: 4
Entering Phase I
MMPC_beggining: 
369CPC size after filtering: 3
['Node183@3', 'Node182@3', 'Node159@2']
Size set for community selection phase will be of maximum: 3
['No

['Node53@3', 'Node306@3', 'Node119@3']
CPC size after filtering: 1

Universe actual size:['Node203@3']

Entering Phase I
MMPC_beggining: 
369
Universe actual size:4


Universe actual size:
6
CPC size before filtering: 6
Size set for community selection phase will be of maximum: 4

CPC size before filtering: 3
Size set for community selection phase will be of maximum: 3
6
['Node203@3', 'Node214@3', 'Node186@4']
['Node186@4', 'Node70@3', 'Node187@4']
CPC size before filtering: 4
Size set for community selection phase will be of maximum: 3
n_set of length: 3
['Node187@4', 'Node70@3', 'Node72@3', 'Node186@4']

Universe actual size:n_set of length: 4
CPC size after filtering: 3

['Node186@4', 'Node214@3', 'Node203@3']
10
CPC size after filtering: 6
CPC size after filtering: 3
CPC size before filtering: 1
CPC size after filtering: 1
['Node163@3', 'Node161@3', 'Node156@3', 'Node155@3', 'Node153@3', 'Node160@3']
Entering Phase I
MMPC_beggining: 
369
['Node188@3']

Universe actual size:
Univers

...

['Node40@3', 'Node37@2', 'Node95@4', 'Node7@3', 'Node55@3']
n_set of length: 5
Entering Phase I
MMPC_beggining: 
369
['Node69@3', 'Node47@3', 'Node49@3', 'Node70@3', 'Node71@3', 'Node72@3', 'Node73@2']
['Node344@2', 'Node343@3', 'Node345@3', 'Node365@3', 'Node366@3', 'Node367@3', 'Node368@3']
Size set for community selection phase will be of maximum: 4
n_set of length: 7
CPC size after filtering: 5
['Node55@3', 'Node7@3', 'Node95@4', 'Node37@2', 'Node40@3']

Universe actual size:
6
CPC size before filtering: 7
['Node314@3', 'Node304@3', 'Node298@4', 'Node297@4', 'Node300@4', 'Node318@4', 'Node325@3']
n_set of length: 7
Entering Phase I
MMPC_beggining: 
369

Universe actual size:n_set of length: 6

20
CPC size before filtering: 12
Size set for community selection phase will be of maximum: 5
['Node364@2', 'Node338@3', 'Node340@3', 'Node341@3', 'Node345@3', 'Node346@3']
CPC size after filtering: 7
['Node331@3', 'Node329@3', 'Node328@3', 'Node267@3', 'Node221@2', 'Node196@2', 'Node330@3']


['Node161@3', 'Node295@2']
n_set of length: 2
n_set of length: 4
CPC size after filtering: 4
['Node293@3', 'Node292@3', 'Node269@3', 'Node271@3', 'Node291@3', 'Node294@3']
n_set of length: 6
['Node64@2', 'Node178@2', 'Node7@3', 'Node8@3']

Universe actual size:
0
CPC size before filtering: 6
Size set for community selection phase will be of maximum: 4
['Node5@3', 'Node7@3', 'Node8@3', 'Node12@3', 'Node13@3', 'Node14@2', 'Node15@3', 'Node49@3', 'Node69@3', 'Node70@3', 'Node71@3', 'Node72@3']
n_set of length: 12
['Node70@3', 'Node47@3', 'Node48@2', 'Node49@3', 'Node69@3', 'Node71@3', 'Node72@3']
n_set of length: 7
CPC size after filtering: 6
['Node295@2', 'Node161@3', 'Node294@3', 'Node291@3', 'Node292@3', 'Node293@3']

Universe actual size:
2
CPC size before filtering: 7
['Node14@2', 'Node1@4', 'Node4@4', 'Node5@3', 'Node12@3', 'Node13@3', 'Node15@3', 'Node18@3', 'Node19@3', 'Node21@4', 'Node22@4', 'Node29@3', 'Node81@3', 'Node82@3']
Size set for community selection phase will be of max

n_set of length: 1
['Node95@4']
n_set of length: 1
CPC size after filtering: 6
['Node7@3', 'Node37@2', 'Node40@3', 'Node55@3', 'Node288@3', 'Node95@4']

Universe actual size:
0
CPC size before filtering: 7
Size set for community selection phase will be of maximum: 4
['Node63@3', 'Node59@4', 'Node39@4']
n_set of length: 3
['Node9@2']
n_set of length: 1
['Node56@3', 'Node41@4', 'Node58@4', 'Node38@4']
n_set of length: 4
CPC size after filtering: 8
['Node39@4', 'Node59@4', 'Node63@3', 'Node9@2', 'Node38@4', 'Node58@4', 'Node41@4', 'Node56@3']
['Node281@4']
n_set of length: 1
['Node42@3', 'Node41@4', 'Node52@3']
n_set of length: 3
['Node91@2']
n_set of length: 1
['Node318@4']
n_set of length: 1
['Node130@3']
n_set of length: 1
['Node35@3']
n_set of length: 1
CPC size after filtering: 8
['Node281@4', 'Node52@3', 'Node41@4', 'Node42@3', 'Node91@2', 'Node318@4', 'Node130@3', 'Node35@3']
CPC size after filtering: 10
['Node142@2', 'Node129@3', 'Node127@3', 'Node126@3', 'Node124@3', 'Node123@3',

['Node322@3']
n_set of length: 1
CPC size after filtering: 8
['Node72@3', 'Node71@3', 'Node70@3', 'Node69@3', 'Node49@3', 'Node5@3', 'Node14@2', 'Node322@3']
['Node16@3']
n_set of length: 1
['Node31@2']
n_set of length: 1
CPC size after filtering: 14
['Node74@2', 'Node72@3', 'Node69@3', 'Node18@3', 'Node15@3', 'Node13@3', 'Node12@3', 'Node8@3', 'Node7@3', 'Node5@3', 'Node4@4', 'Node21@4', 'Node16@3', 'Node31@2']
Node101@2
Node102@2
Node104@2
Node105@2
Node110@2
Node111@2
Node117@2
Node11@2
Node120@2
Node122@2
Node125@2
Node128@2
Node131@2
Node135@2
Node138@2
Node139@2
Node141@2
Node142@2
Node147@2
Node148@2
Node14@2
Node154@2
Node157@2
Node159@2
Node162@2
Node165@2
Node168@2
Node172@2
Node175@2
Node176@2
Node178@2
Node179@2
Node17@2
Node184@2
Node185@2
Node191@2
Node194@2
Node196@2
Node199@2
Node202@2
Node205@2
Node209@2
Node20@2
Node212@2
Node213@2
Node215@2
Node216@2
Node221@2
Node222@2
Node228@2
Node231@2
Node233@2
Node236@2
Node239@2
Node242@2
Node246@2
Node249@2
Node24@2
Node250@2

Size set for community selection phase will be of maximum: 2
['Node36@2', 'Node35@3']
n_set of length: 2
CPC size after filtering: 2

Universe actual size:['Node35@3', 'Node36@2']


Universe actual size:3
Entering Phase I
CPC size before filtering: 1
MMPC_beggining: 
369

1
['Node118@3']

Universe actual size:CPC size before filtering: 3

Universe actual size:

0
Size set for community selection phase will be of maximum: 3
CPC size before filtering: 6
['Node35@3', 'Node36@2', 'Node356@4']
2
Size set for community selection phase will be of maximum: 4
n_set of length: 2
CPC size before filtering: 2
n_set of length: 3
Size set for community selection phase will be of maximum: 2
['Node118@3', 'Node119@3']
CPC size after filtering: 3
['Node356@4', 'Node36@2', 'Node35@3']
CPC size after filtering: 2
['Node119@3', 'Node118@3']

Universe actual size:

Universe actual size:0

CPC size before filtering: 4
1
CPC size before filtering: 3
Size set for community selection phase will be of maximum: 